In [1]:
import pandas as pd
df = pd.read_csv(r'Yield_Weather.csv')
df.head(3)

,Unnamed: 0,Year,Location,Production (mt),Area (ha),Yield (mt/ha),weathercode (wmo code),temperature_2m (°C),relativehumidity_2m (%),dewpoint_2m (°C),...,cloudcover (%),et0_fao_evapotranspiration (mm),vapor_pressure_deficit (kPa),windspeed_10m (km/h),winddirection_10m (°),windgusts_10m (km/h),soil_temperature_28_to_100cm (°C),soil_moisture_28_to_100cm (m³/m³),direct_radiation (W/m²),diffuse_radiation (W/m²)
0,0,1990,ZAMBOANGA DEL NORTE,41.14,91.0,0.452088,3,23.486279,86.200457,20.899235,...,70.154680,0.153233,0.435739,6.732808,144.312329,19.894932,25.227021,0.401853,151.119064,70.720548
1,1,1991,ZAMBOANGA DEL NORTE,53.28,108.0,0.493333,3,23.378607,86.403995,20.856210,...,71.524658,0.153091,0.422427,6.505616,138.691096,19.141393,25.074669,0.404063,147.845890,75.080251
2,2,1992,ZAMBOANGA DEL NORTE,71.34,114.0,0.625789,3,23.253153,86.111225,20.664014,...,66.843010,0.154016,0.430523,5.957605,143.278233,18.114481,25.086202,0.399902,149.374203,74.632628


In [2]:
import numpy as np

# name of the label (can be seen in the dataframe)
label = 'Yield (mt/ha)'

# list with feature names (V1, V2, V3, ...)
features = df.columns.tolist()
features.remove(label)

In [3]:
df

,Unnamed: 0,Year,Location,Production (mt),Area (ha),Yield (mt/ha),weathercode (wmo code),temperature_2m (°C),relativehumidity_2m (%),dewpoint_2m (°C),...,cloudcover (%),et0_fao_evapotranspiration (mm),vapor_pressure_deficit (kPa),windspeed_10m (km/h),winddirection_10m (°),windgusts_10m (km/h),soil_temperature_28_to_100cm (°C),soil_moisture_28_to_100cm (m³/m³),direct_radiation (W/m²),diffuse_radiation (W/m²)
0,0,1990,ZAMBOANGA DEL NORTE,41.14,91.0,0.452088,3,23.486279,86.200457,20.899235,...,70.154680,0.153233,0.435739,6.732808,144.312329,19.894932,25.227021,0.401853,151.119064,70.720548
1,1,1991,ZAMBOANGA DEL NORTE,53.28,108.0,0.493333,3,23.378607,86.403995,20.856210,...,71.524658,0.153091,0.422427,6.505616,138.691096,19.141393,25.074669,0.404063,147.845890,75.080251
2,2,1992,ZAMBOANGA DEL NORTE,71.34,114.0,0.625789,3,23.253153,86.111225,20.664014,...,66.843010,0.154016,0.430523,5.957605,143.278233,18.114481,25.086202,0.399902,149.374203,74.632628
3,3,1993,ZAMBOANGA DEL NORTE,78.88,123.0,0.641301,3,23.086187,88.171575,20.931313,...,72.979110,0.147360,0.360022,5.702637,151.702854,17.351541,24.700765,0.450554,141.238014,75.799315
4,4,1994,ZAMBOANGA DEL NORTE,85.71,128.0,0.669609,3,23.100217,88.401142,20.990674,...,76.482534,0.144709,0.352926,5.919954,154.063813,18.403128,24.679966,0.438253,139.645320,73.633562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,952,2018,TAWI-TAWI,0.00,0.0,NaN,1,25.101781,81.959703,21.787637,...,43.909932,0.164605,0.577837,13.415742,174.348288,21.051290,26.616324,0.000000,164.635959,71.210502
953,953,2019,TAWI-TAWI,0.00,0.0,NaN,1,25.174361,81.307420,21.708082,...,36.225799,0.174063,0.600727,13.434920,150.258790,20.562249,26.571689,0.000000,184.635274,65.579452
954,954,2020,TAWI-TAWI,0.00,0.0,NaN,1,25.272291,80.921790,21.740745,...,43.682491,0.165880,0.617793,12.067725,143.826617,19.564526,26.978802,0.000000,165.257628,68.485087
955,955,2021,TAWI-TAWI,0.00,0.0,NaN,1,25.110696,81.587671,21.717591,...,47.829110,0.159237,0.591193,12.753904,179.280365,20.301929,26.899030,0.000000,150.401142,73.859817


In [4]:
from scipy.stats import pointbiserialr
from math import sqrt

def getMerit(subset, label):
    k = len(subset)

    # average feature-class correlation
    rcf_all = []
    for feature in subset:
        coeff = pointbiserialr( df[label], df[feature] )
        rcf_all.append( abs( coeff.correlation ) )
    rcf = np.mean( rcf_all )

    # average feature-feature correlation
    corr = df[subset].corr()
    corr.values[np.tril_indices_from(corr.values)] = np.nan
    corr = abs(corr)
    rff = corr.unstack().mean()

    return (k * rcf) / sqrt(k + k * (k-1) * rff)

In [5]:
subset = ['temperature_2m (°C)', 'relativehumidity_2m (%)', 'dewpoint_2m (°C)', 'rain (mm)']
corr = df[subset].corr()
corr

,temperature_2m (°C),relativehumidity_2m (%),dewpoint_2m (°C),rain (mm)
temperature_2m (°C),1.000000,-0.689448,0.969706,-0.427127
relativehumidity_2m (%),-0.689448,1.000000,-0.493857,0.603944
dewpoint_2m (°C),0.969706,-0.493857,1.000000,-0.315020
rain (mm),-0.427127,0.603944,-0.315020,1.000000


In [6]:
corr.values[np.tril_indices_from(corr.values)] = np.nan
corr

,temperature_2m (°C),relativehumidity_2m (%),dewpoint_2m (°C),rain (mm)
temperature_2m (°C),NaN,-0.689448,0.969706,-0.427127
relativehumidity_2m (%),NaN,NaN,-0.493857,0.603944
dewpoint_2m (°C),NaN,NaN,NaN,-0.315020
rain (mm),NaN,NaN,NaN,NaN


In [7]:
best_value = -1
best_feature = ''
for feature in features:
    coeff = pointbiserialr( df[label], df[feature] )
    abs_coeff = abs( coeff.correlation )
    if abs_coeff > best_value:
        best_value = abs_coeff
        best_feature = feature

print("Feature %s with merit %.4f"%(best_feature, best_value))

ValueError: array must not contain infs or NaNs